## Check GPU

In [ ]:
import pandas as pd
!nvidia-smi --query-gpu=timestamp,name,\
utilization.gpu,utilization.memory,\
memory.total,memory.free,memory.used --format=csv > gpu.csv
display(pd.read_csv('gpu.csv', index_col=False))

print('\nGPU PRIO:\nK90 < Tesla T4 < P100')

## Allow to SSH into this session


In [ ]:
# Install useful stuff
! apt install --yes ssh screen nano htop ranger git > /dev/null
# SSH setting
! echo "root:johlucfre" | chpasswd
! echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
! echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! service ssh restart > /dev/null
# Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
# Run ngrok
authtoken = "1lsUxIKKMaw24jCJb8wgaDBgdX3_6fVMNEGJdEgwzc7ajkrKy"
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
! sleep 3
# Get the address for SSH
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh
print(str_ssh)
print('pw: johlucfre')

## Install Repo and Requirements

In [ ]:
!ls
!echo "Current directory: $PWD"
![ -d "NLP_commonsense" ] && echo "Directory NLP_commonsense exists." || git clone https://github.com/FredericOdermatt/NLP_commonsense.git

In [ ]:
pip install -r NLP_commonsense/requirements.txt

## Train
Note that to train on a batch-size of 16 you need to have a GPU with around 16GB of GPU RAM (see first cell)

In [ ]:
# trained model stored at /content/$OUTDIR
%cd /content/NLP_commonsense/JUSTers
OUTDIR='/content/colab_trained'
!mkdir -p $OUTDIR
!ls
!./train.sh $OUTDIR 16 5 5

## Evaluation

Calculate Scores for the output of the KaLM model stored at data100/kalm.csv

In [ ]:
%cd /content/NLP_commonsense/
!./evaluate.sh data100/references_complete.csv data100/kalm.csv

## Visualization

Create plots showing correlations between scoring functions.
**Note**: Before executing this cell you must evaluate the output using the cell above.

In [ ]:
!python Visualization/visualize_scores.py
from IPython.display import Image
Image(filename='pairplot.png') 

## Copy to Leonhard

Execute this command on leonhard and care to look at port number -p (execute cell below)

`rsync -arvz -e 'ssh -p 17471' --progress root@4.tcp.ngrok.io:/content/colab_trained $HOME`



In [ ]:
print(str_ssh)
print('pw: johlucfre')